## Attention Is All You Need<br><br>
Transformer 모델이 등장했으며, 이 모델은 오로지 attention 메카니즘에만 의존했다.<br><br>
- WMT 2014 English to German: 28.4 BLEU score<br>
- WMT 2014 English to French: 41.8 BLEU score (SOTA)<br><br><br>
#### Model Architecture<br><br>
인코더는 input sequence $(x_{1},...,x_{n})$를 받아서 map하면서 $z$로, continuous representaion, $(z_{1},...,z_{n})$으로 변경한다<br><br>

디코더는 이 $z$를 받아서, output sequence $(y_{1},...,y_{m})$을 한 번에 한 개씩 생성한다. 각 스텝마다 모델은 auto-regressive하게 움직인다.<br><br>

트랜스포머 모델은 stacked self-attention과 point-wise, fully-connected를 인코더와 디코더 모두에 사용한 위의 아키텍쳐를 쓴다.<br><br>

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FR8wLW%2Fbtq1AGlF1o5%2FdDhkrrvZfAY1Pp6vubbUbk%2Fimg.png)<br><br>
Encoder(왼쪽) - Decoder(오른쪽) 구조<br>
(이는 훗날 인코더는 BERT로, 디코더는 GPT로 진화하게 된다.)<br><br>

### Encoder<br><br>
N=6개의 동일한 레이어들의 stack으로 구성되어 있다. 각 레이어는 두 개의 서브 레이어가 있는데, 첫 번째는 multi-head self-attention 메커니즘이고, 두 번째는 position-wise fully connected feedforward network이다.  <br><br>

두 개의 서브 레이어 주변에는 residual connection이 있고, 이어서 layer normalization이 나온다.<br><br>
각 서브 레이어의 아웃풋은 $LayerNorm(x+Sublayer(x))$이다. 여기서 Sublayer는 해당 레이어 자체적으로 적용된 함수다.<br><br>

이 residual connection에서, 모델의 모든 서브 레이어들은, 그리고 embedding 레이어 역시도 아웃풋의 차원이 $d_{model}=512$이다. <br><br><br>

### Decoder<br><br>
디코더 역시도 동일한 레이어들의 N=6 stack으로 이루어져 있다. 각 인코더 레이어의 2개의 서브 레이어들에 더해서, 디코더는 3번째의 서브 레이어가 있다. 이게 multi-head attention을 인코더의 아웃풋에 하는 것이다.<br><br>
인코더랑 비슷하게 각 서브 레이어마다 residual connection이 있고, 이후에 layer norm이 온다. 디코더 stack의 self-attention 서브 레이어를 수정해서, 뒤의 위치에서부터 오는 attention을 방지한다.<br><br>

### Output<br><br>
어텐션 함수는 쿼리와 키-밸류 페어들의 세트를 매핑하는 것으로 묘사할 수 있다. 쿼리, 키, 밸류, 아웃풋 전부는 벡터들이다.<br><br>

아웃풋은 밸류들의 weighted sum으로 계산된다. 이 weight(가중치)은 키(key)에 상응하는 쿼리(query)의 compatibility 함수에 의해, 각 value에 적용된 weight 값이 계산된다.<br><br><br>

## Scaled Dot-product Attention<br><br>
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FCcf0m%2Fbtq1xmPyX9Y%2FvYKuXKbKKpgRdJQYZGCMg0%2Fimg.png)<br><br>


- scaling factor: $\frac{1}{\sqrt{d_{k}}}$<br>
- dot product: $(QK^{T})$<br>
- softmax: softmax$(\frac {QK^{T}}{\sqrt d_{k}})$
- masking (option): 수식에는 없음<br><br>
- embedding dim of query: $d_{k}$<br>
- embedding dim of key: $d_{k}$<br>
- embedding dim of value: $d_{v}$<br>
- dim of output: $d_{v}$<br><br><br>

아웃풋들은 concat되고, 다시 한 번 project되고, final valuues를 꺼내놓는다 (위의 그림에서는 Linear부분이 projection일 듯)

<br><br>
## Multi-Head Attention<br><br>
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbEtNir%2Fbtq1QmOgrJL%2FfndhYjfeWINHIYbKBafxf0%2Fimg.png)<br><br>

Multi-Head Attention은 모델이, 다른 포지션의 다른 representation subspaces로부터의 정보에 jointly attend하게 해준다.<br><br>

- $MultiHead(Q,K,V) = Concat(head_{1},...,head_{n})W^{0}$<br>

- $head_{i}=Attention(QW^{Q}_{i},KW^{K}_{i},VW^{V}_{i})$ <br><br>
$W_{Q}_{i} \in R^{d_{model}xd_{k}}

